In [689]:
#Imports
import numpy as np
import pandas as pd
import plotly.express as px

In [690]:
#Sampling Data from CSV
df = pd.read_csv("Testing Data/3-4-2025/2025_03_04_21_51_55.0 (LOX 7).csv")

fire_start_index = df[df["MACHINE"] == 2].index.min()

sample_start_time = df.loc[fire_start_index-1, "PACKET_TIMESECONDS"]
sample_end_time = sample_start_time + 8

sample = df[(df["PACKET_TIMESECONDS"] >= sample_start_time) & (df["PACKET_TIMESECONDS"] <= sample_end_time)]

time_since_fire = sample["PACKET_TIMESECONDS"] - sample["PACKET_TIMESECONDS"].iloc[0]

sample["TIME_SINCE_FIRE"] = time_since_fire


C:\Users\goose\AppData\Local\Temp\ipykernel_1632\1704028559.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [691]:
#Plotting

press_fig = px.line(sample, x = "TIME_SINCE_FIRE", y = ["NITROT", "LOXT", "LOXISO", "LOXMAN"], title = "System Pressures vs Time") #Change according to what you'd like

press_fig.update_layout(
    xaxis_title="Time (seconds)",
    yaxis_title="Pressure (psi)",
    legend_title="Sensor",
)

flow_fig = px.line(sample, x = "TIME_SINCE_FIRE", y = "KEROFLOW", title = "System Flow Rates vs Time")

flow_fig.update_layout(
    xaxis_title="Time (seconds)",
    yaxis_title="Flow Rate (L/min)",
    legend_title="Sensor",
)

press_fig.show()
flow_fig.show()

In [692]:
#Averaging Measurements
averaging_sample = sample[(sample["TIME_SINCE_FIRE"] >= 2) & (sample["TIME_SINCE_FIRE"] <= 4)]

NITROT_ave = np.mean(averaging_sample["NITROT"])

LOXISO_ave = np.mean(averaging_sample["LOXISO"])
LOXT_ave = np.mean(averaging_sample["LOXT"])
LOXMAN_ave = np.mean(averaging_sample["LOXMAN"])

KEROISO_ave = np.mean(averaging_sample["KEROISO"])
KEROT_ave = np.mean(averaging_sample["KEROT"])
KEROMAN_ave = np.mean(averaging_sample["KEROMAN"])

FLOW_ave = np.mean(averaging_sample["KEROFLOW"])/(28.317*60) #convert L/min -> ft^3/s

print(f"Averaged NITROT: {NITROT_ave} \n")
print(f"Averaged LOXISO: {LOXISO_ave}")
print(f"Averaged LOXT: {LOXT_ave}")
print(f"Averaged LOXMAN: {LOXMAN_ave} \n")
print(f"Averaged KEROISO: {KEROISO_ave}")
print(f"Averaged KEROT: {KEROT_ave}")
print(f"Averaged KEROMAN: {KEROMAN_ave} \n")
print(f"Averaged FLOW: {FLOW_ave}")

Averaged NITROT: 876.3169543666941 

Averaged LOXISO: 517.6562101468686
Averaged LOXT: 466.02885560117494
Averaged LOXMAN: 234.2969646201164 

Averaged KEROISO: -108.46614278250141
Averaged KEROT: -1.9748044293675129
Averaged KEROMAN: -1.682345029026936 

Averaged FLOW: 0.03435267159271581


In [693]:
#Setting dP's
dP_isolation_line_LOX = (LOXISO_ave - LOXT_ave)
dP_main_line_LOX = (LOXT_ave - LOXMAN_ave)
dP_injector_LOX = (LOXMAN_ave)

dP_isolation_line_KERO = (KEROISO_ave - KEROT_ave)
dP_main_line_KERO = (KEROT_ave - KEROMAN_ave)
dP_injector_KERO = (KEROMAN_ave)

#CdA Calculation
rho_water = 1.940 #slugs/ft^3
rho_nitrogen_LOX = 0.0001485412225*LOXISO_ave #slugs/ft^3 (Linear fit to N2 Density Data at T = 298K)
rho_nitrogen_KERO = 0.0001485412225*KEROISO_ave #slugs/ft^3 (Linear fit to N2 Density Data at T = 298K)

#Mass flow of Nitrogen in LOX/Kero Lines
mass_flow_nitrogen_LOX = FLOW_ave * rho_nitrogen_LOX #slugs/s
mass_flow_nitrogen_KERO = FLOW_ave * rho_nitrogen_KERO #slugs/s

#Mass flow of Water in LOX/Kero Lines
mass_flow_water_LOX = FLOW_ave * rho_water #slugs/s
mass_flow_water_KERO = FLOW_ave * rho_water #slugs/s

#LOX Side CdA's
CdA_isolation_line_LOX = mass_flow_nitrogen_LOX/(np.sqrt(2*rho_nitrogen_LOX*dP_isolation_line_LOX*144)) #144 for psi -> psf
CdA_main_line_LOX = mass_flow_water_LOX/(np.sqrt(2*rho_water*dP_main_line_LOX*144)) #144 for psi -> psf
CdA_injector_LOX = mass_flow_water_LOX/(np.sqrt(2*rho_water*dP_injector_LOX*144)) #144 for psi -> psf
CdA_combined_LOX = mass_flow_water_LOX/(np.sqrt(2*rho_water*(dP_main_line_LOX+dP_injector_LOX)*144)) #combined main line + injector

#Kero Side CdA's
CdA_isolation_line_KERO = mass_flow_nitrogen_KERO/(np.sqrt(2*rho_nitrogen_KERO*dP_isolation_line_KERO*144)) #144 for psi -> psf
CdA_main_line_KERO = mass_flow_water_KERO/(np.sqrt(2*rho_water*dP_main_line_KERO*144)) #144 for psi -> psf
CdA_injector_KERO = mass_flow_water_KERO/(np.sqrt(2*rho_water*dP_injector_KERO*144)) #144 for psi -> psf
CdA_combined_KERO = mass_flow_water_KERO/(np.sqrt(2*rho_water*(dP_main_line_KERO+dP_injector_KERO)*144)) #combined main line + injector

print(f"TRUST ONLY IF LINE WAS ACTUALLY PART OF TEST: \n")

print(f"CdA_isolation_line_LOX: {CdA_isolation_line_LOX}")
print(f"CdA_main_line_LOX: {CdA_main_line_LOX}")
print(f"CdA_injector_LOX: {CdA_injector_LOX}")
print(f"CdA_combined_LOX: {CdA_combined_LOX} \n")

print(f"CdA_isolation_line_KERO: {CdA_isolation_line_KERO}")
print(f"CdA_main_line_KERO: {CdA_main_line_KERO}")
print(f"CdA_injector_KERO: {CdA_injector_KERO}")
print(f"CdA_combined_KERO: {CdA_combined_KERO} \n")


TRUST ONLY IF LINE WAS ACTUALLY PART OF TEST: 

CdA_isolation_line_LOX: 7.812113582504237e-05
CdA_main_line_LOX: 0.00018521332820713068
CdA_injector_LOX: 0.00018419668401253647
CdA_combined_LOX: 0.00013060467854521512 

CdA_isolation_line_KERO: -2.489876041106797e-05
CdA_main_line_KERO: nan
CdA_injector_KERO: nan
CdA_combined_KERO: nan 



C:\Users\goose\AppData\Local\Temp\ipykernel_1632\3059034410.py:31: RuntimeWarning:

invalid value encountered in sqrt

C:\Users\goose\AppData\Local\Temp\ipykernel_1632\3059034410.py:32: RuntimeWarning:

invalid value encountered in sqrt

C:\Users\goose\AppData\Local\Temp\ipykernel_1632\3059034410.py:33: RuntimeWarning:

invalid value encountered in sqrt



In [694]:
#Expected Set Pressures
rho_LOX = 2.128531 #slugs/ft^3
rho_KERO = 1.59106 #slugs/ft^3

chamber_pressure = 565 #psi
mass_flow_total = 0.10148073 #slugs/s
OF_Ratio = 1.8

#Using propellant mass flows and CdA's to obtain tank pressures
mass_flow_LOX = mass_flow_total/((1/OF_Ratio)+1)
mass_flow_KERO = mass_flow_total/(OF_Ratio + 1)

expected_tank_press_LOX = ((mass_flow_LOX/CdA_combined_LOX)**2)/(2*rho_LOX*144) + chamber_pressure
expected_tank_press_KERO = ((mass_flow_KERO/CdA_combined_KERO)**2)/(2*rho_KERO*144) + chamber_pressure

#Using nitrogen mass flows and CdA's to obtain reg pressures
expected_rho_nitrogen_LOX = 0.0001485412225*expected_tank_press_LOX
expected_rho_nitrogen_KERO = 0.0001485412225*expected_tank_press_KERO

expected_mass_flow_nitrogen_LOX = (mass_flow_LOX*expected_rho_nitrogen_LOX)/(rho_LOX)
expected_mass_flow_nitrogen_KERO = (mass_flow_KERO*expected_rho_nitrogen_KERO)/(rho_KERO)

reg_press_LOX = ((expected_mass_flow_nitrogen_LOX/CdA_isolation_line_LOX)**2)/(2*expected_rho_nitrogen_LOX*144) + expected_tank_press_LOX
reg_press_KERO = ((expected_mass_flow_nitrogen_KERO/CdA_isolation_line_KERO)**2)/(2*expected_rho_nitrogen_KERO*144) + expected_tank_press_KERO

print(f"Set LOX Tank to: {expected_tank_press_LOX}")
print(f"Set KERO Tank to: {expected_tank_press_KERO} \n")

print(f"Set LOX Reg to: {reg_press_LOX}")
print(f"Set KERO Reg to: {reg_press_KERO} \n")

Set LOX Tank to: 972.011027627337
Set KERO Tank to: nan 

Set LOX Reg to: 1049.1768082168624
Set KERO Reg to: nan 

